Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Advanced Exercise in Data Science: Kalman Fidlers

In this notebook, we'll implement an [Extended Kalman Filter](https://en.wikipedia.org/wiki/Extended_Kalman_filter) for terrain-based navigation using GPS, an altimeter, and INS. The code below is "pseudocode" and will not run out of the box because some of the inputs (like the terrain map) need to be supplied and not all variables are explicitly defined. Variable names are descriptive to enable filling in the details... it's an exercise for the dedicated student!

## Calculate Map Gradients
* Load map data
* Create an interpolation function 

In [ ]:
### Get Gradient Jacobians (Change in h(x) i.e.  ground level/ Change in x/y)
grad_lat = (np.gradient(map_terrain, axis = 0))/75

grad_lon = (np.gradient(map_terrain, axis = 1))/75

grid_points = np.array(list(product(map_lat_range, map_lon_range)))
map_grad_stack_lat = grad_lat.reshape(-1,1)
map_grad_stack_lon = grad_lon.reshape(-1,1)
# lat, lon
func_map_grad_lat = LinearNDInterpolator( \
                                  np.vstack((grid_points[:,0], grid_points[:,1])).T, \
                                  map_grad_stack_lat, \
                                  fill_value=np.nan, \
                                  rescale=False)

func_map_grad_lon = LinearNDInterpolator( \
                                  np.vstack((grid_points[:,0], grid_points[:,1])).T, \
                                  map_grad_stack_lon, \
                                  fill_value=np.nan, \
                                  rescale=False)

## Terrain Altimeter Sensor

In [ ]:
# Load Altimeter data
z_alt = # LOAD ALTIMETER DATA

In [ ]:
# Filtering utilities
import numpy as np
from scipy.fftpack import fft
from scipy import signal


def median_filter(s, f_size):
  return signal.medfilt(s, f_size)

def freq_filter(s, f_size, cutoff):
  lpf=signal.firwin(f_size, cutoff, window='hamming')
  return signal.convolve(s, lpf, mode='same')

def comb_filter(s, f_size, cutoff, fs):
  median_s=median_filter(s, f_size)
  return freq_filter(median_s, f_size, cutoff/fs)

In [ ]:
fs=200
f_size = 55
cutoff = 10
g = 9.80665

z_alt_lp = comb_filter(z_alt.flatten(), f_size, cutoff, fs)

## Set Up Navigation Filter

[pos, vel, ins_drift, ins_drift_rate]

In [ ]:
# Predict and Update Functions
def predict(x, P, F, Q):
  x = F @ x
  P = F @ P @ F.T + Q

  return x, P

def update(x, P, z, H, R, debug = False):
  dim_x = len(x)
  y = z - H @ x
  S = H @ P @ H.T + R
  K = P @ H.T @ np.linalg.pinv(S)
  x = x + K @ y
  P = (np.eye(dim_x) - K @ H) @ P
  y2 = z - H @ x    
  if debug:
    return x, P, y, y2
  else:
    return x, P

def update_terrain(x, P, z, H, R, func_map):
  dim_x = len(x)

  # Get current LLA
  z_pred = func_map(x[0], x[1])  

  # Update
  y = z - z_pred
  S = H @ P @ H.T + R  
  K = P @ H.T @ np.linalg.pinv(S)

  x = x + K @ y
  P = (np.eye(dim_x) - K @ H) @ P
  return x, P

In [ ]:
# Process Model
F = np.array([[1, dt, 0,  0, 0, 0, 0, 0],
              [0,  1, 0,  0, 0, 0, 0, 0],
              [0,  0, 1, dt, 0, 0, 0, 0],
              [0,  0, 0,  1, 0, 0, 0, 0],
              [0,  0, 0,  0, 1, dt, 0, 0],
              [0,  0, 0,  0, 0, 1, 0, 0]
              [0,  0, 0,  0, 0, 0, 1, dt],
              [0,  0, 0,  0, 0, 0, 0, dt]])

# Measurement Models
H_vel = np.array([[0, 1, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0]])
H_gps = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0, 0]])


In [ ]:
# Logging arrays
x_mat, P_mat, residual_mat, grad_mat = [], [], [], []

# Initialize filter
ind_sim = 1000

# Initial Conditions
x0 = np.array([0, 0, 0, 0, 0, 0])
P0 = np.diag([100**2, 100**2, 10**2, 10**2, 10**2, 10**2, 10**2, 10**2])

# Measurement noise
R_vel = np.diag([10, 10])
R_gps = np.diag([10, 10])
R_alt = np.diag([100])

# Process Noise
Q = np.diag([10**2, 10**2, 1**2, 1**2, .1**2, .1**2, .01**2, .01**2])

for i in range(ind_test):
  z_vel = # velocity data
  z_gps = # gps data
  z_alt = # Filtered altimeter data
  
  # Obtain Jacobian from Terrain Map
  dzdx = func_map_grad_x(x[0], x[1])
  dzdy = func_map_grad_y(x[0], x[1])
  H_map = np.array([[dzdx[0], 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, dzdy[0], 0, 0, 0, 0, 0]])

  ## Update
  x, P = update(x, P, z_vel, H_vel_bias, R_vel)

  if i % c_rate == 0 and flag_sensor:
    if sensor == 'gps':
      x, P = update(x, P, z_gps, H_gps, R_gps)
    elif sensor == 'terrain':
      x, P, y_pre, y_post = update_terrain(x, P, z_alt, H_alt, R_alt, func_map) 
 
  ## Log
  x_mat.append(x),  P_mat.append(P)

  ## Predict
  x, P = predict(x, P, F, Q)

## Plot Results

In [ ]:
plt.figure(figsize = (16,4))
plt.subplot(1,2,1)
plt.plot(x_mat[:,0], ll_mat[:,1])
plt.xlabel('x'), plt.ylabel('y')
plt.subplot(1,2,2)
pos_err = np.linalg.norm(x_true - x)
plt.plot(abs(pos_err))
